In [40]:
import pandas as pd
import glob
import re
from datetime import datetime, timedelta
from utils import read_file,transform_files,clean_column_names,all_history

In [63]:
file = pd.read_excel('./data rl for sale.xlsx', dtype={'LOAN_NO': str})

In [64]:
contract = pd.read_csv('./contract.csv', dtype={'MOBILE_PHONE_NO': str,'CONTRACT_NO':str})

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_3632\2985140201.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  contract = pd.read_csv('./contract.csv', dtype={'MOBILE_PHONE_NO': str,'CONTRACT_NO':str})


In [65]:
contract

,CONTRACT_NO,NATIONAL_ID,CURRENT_DUE_AMT,NAME,SURNAME,MOBILE_PHONE_NO,RESIDENT_ADDRESS
0,7240670102985111,1500900023779,771.00,ประเสริฐ,มุกศรีมา,0847393497,41/1 หมู่ที่ 3 ต. โป่งน้ำร้อน อ. ฝาง จ. เชีย...
1,7240670102985121,5311000125074,331.23,ณัชชา,สุทธะสนธิ์,0819152385,89 หมู่ที่ 15 ต. แสลงพัน อ. ลำปลายมาศ จ. บุร...
2,7240670103076979,1670400170461,NaN,กิดานันท์,สังสนา,0839922858,224 หมู่ที่ 9 ต. ตาดกลอย อ. หล่มเก่า จ. เพชร...
3,7240870103520832,1670400170461,NaN,กิดานันท์,สังสนา,0839922858,224 หมู่ที่ 9 ต. ตาดกลอย อ. หล่มเก่า จ. เพชร...
4,7240670103114698,3500400398249,NaN,เบ็ญจวรรณ์,กันธวงค์,0625394871,51 หมู่ที่ 2 ต. เมืองคอง อ. เชียงดาว จ. เชีย...
...,...,...,...,...,...,...,...
5593840,7250470105196401,3400700465929,NaN,NaN,NaN,NaN,NaN
5593841,7221230100089987,3331000951491,NaN,NaN,NaN,NaN,NaN
5593842,7250470105196496,1719900202769,NaN,NaN,NaN,NaN,NaN
5593843,7221170100027530,3650400492121,NaN,NaN,NaN,NaN,NaN


In [66]:
file

,LOAN_NO
0,7250330104975190
1,7250130104602714
2,7241030104083572
3,7241030104134167
4,7241030104022567
...,...
857,7250430105101326
858,7250430105110631
859,7250430105127468
860,7250430105133982


In [67]:
df = file.merge(contract,left_on='LOAN_NO',right_on='CONTRACT_NO', how='left')

In [68]:
df.to_excel('./outputloaninfo.xlsx',index='False')